#### Q1. What is the difference between `__getattr__` and `__getattribute__`?
**Ans:** `__getattr__` is invoked if the attribute wasn't found the usual ways and is good for implementing a fallback for missing attributes.

Whereas the `__getattribute__` is invoked before looking at the actual attributes on the object. Thus, we have to use it more consciously, otherwise very easily we can end up in infinite recursions.

In [6]:
#Example
class Count():
    def __init__(self,min,max):
        self.min=min
        self.max=max

obj1 = Count(1,10)
print(obj1.min)
print(obj1.max)
print(obj1.current) #gives AttributeError because object has no attribute 'current'

1
10


AttributeError: 'Count' object has no attribute 'current'

In [7]:
#Example with __getattr__
class Count:
    def __init__(self,min,max):
        self.min=min
        self.max=max    

    def __getattr__(self, item):
        self.__dict__[item]=0
        return 0

obj1 = Count(1,10)
print(obj1.min)
print(obj1.max)
print(obj1.current)
print(obj1.new_current)

1
10
0
0


Python will call `__getattr__` method whenever we request an attribute that hasn't already been defined.<br/>
In above example whenever we try to call an attribute which doesn't exist, python creates that attribute and sets it to integer value 0.

In [17]:
#Example with __getattribute__
class Count:
    def __init__(self,min,max):
        self.min=min
        self.max=max    

    def __getattribute__(self, item):
        if item.startswith('cur'):
            raise AttributeError(f'Count object has no attribute {item}.')
        return super().__getattribute__(item)

obj1 = Count(1,10)
print(obj1.min)
print(obj1.max)
print(obj1.current)


1
10


AttributeError: Count object has no attribute current.

If we have `__getattribute__` method in class, python invokes this method for every attribute regardless whether it exists or not.<br/>
In above example, whenever attribute is accessed if attribute starts with substring 'cur' then python raises AttributeError exception.<br/> Otherwise it returns that attribute.<br/>
In order to avoid infinite recursion, its implementation should always call the base class method, ie; object.`__getattribute__`(self, name) <br/>
or super().`__getattribute__`(item) 

In [18]:
#Example with both __getattr__ and __getattribute__
class Count:
    def __init__(self,min,max):
        self.min=min
        self.max=max    

    def __getattr__(self, item):
        self.__dict__[item]=0
        return 0
    
    def __getattribute__(self, item):
        if item.startswith('cur'):
            raise AttributeError(f'Count object has no attribute {item}.')
        return super().__getattribute__(item)
    

obj1 = Count(1,10)
print(obj1.min)
print(obj1.max)
print(obj1.current)

1
10
0


If the class contain both getattr and getattribute , then `__getattribute__` is called first. But if `__getattribute__` raises AttributeError exception then the exception will be ignored and `__getattr__` method will be invoked.

#### Q2. What is the difference between properties and descriptors?
**Ans:** The differences between Properties and Descriptors is:

**Properties:** With Properties we can bind getter, setter and delete functions together with an attribute name, using the built-in property function or `@property` decorator. When we do this, each reference to an attribute looks like simple, direct access, but involes the appropriate function of the object. <br/>
Syntax: property(fget, fset, fdel, doc)

**Descriptor:** With Descriptor we can bind getter, setter and delete functions into a seperate class. we then assign an object of this class to the attribute name in our main class. When we do this, each reference to an attribute looks like simple, direct access but invokes an appropriate function of descriptor object.

A Python object that implements any of the following methods is a descriptor.

`__get__`(self, obj, type=None)<br>
`__set__`(self, obj, value)<br>
`__delete__`(self, obj)<br>

The above methods are also known as descriptor protocols and the meanings of the parameters in these methods are as follows.

self is the currently defined descriptor object instance.<br>
obj is the instance of the object that the descriptor will act on.<br>
type is the type of the object on which the descriptor acts (i.e., the class it belongs to).

In [38]:
#Example of Descriptor
class Descriptor:

    def __get__(self, instance, owner):
        if instance is None:
            print('__get__(): Accessing x from the class', owner)
            return self
        
        print('__get__(): Accessing x from the object', instance)
        return 'X from descriptor'

    def __set__(self, instance, value):
        print('__set__(): Setting x on the object', instance)
        instance.__dict__['_x'] = value

class Foo:
    x = Descriptor()

print(Foo.x)

__get__(): Accessing x from the class <class '__main__.Foo'>


In [39]:
foo = Foo()
print(foo.x)

__get__(): Accessing x from the object <__main__.Foo object at 0x0000017492915C10>
X from descriptor


In [40]:
foo.x = 1

__set__(): Setting x on the object <__main__.Foo object at 0x0000017492915C10>


In [41]:
print(foo.x)

__get__(): Accessing x from the object <__main__.Foo object at 0x0000017492915C10>
X from descriptor


In [31]:
#Another Example of Descriptor
class Celsius( object ):
    def __init__( self, value=0.0 ):
        self.value= float(value)
    def __get__( self, instance, owner ):
        return self.value
    def __set__( self, instance, value ):
        self.value= float(value)
        
class Farenheit( object ):
    def __get__( self, instance, owner ):
        return round(instance.celsius * 9 / 5 + 32,2)
    def __set__( self, instance, value ):
        instance.celsius= round((float(value)-32) * 5 / 9,2)

class Temperature( object ):
    celsius= Celsius()
    farenheit= Farenheit()

oven= Temperature()
oven.farenheit= 100
print(f'{oven.farenheit} Farenheit is equivalent to  {oven.celsius} Celsius')
celsius= oven.celsius
oven.celsius = celsius
print(f'{oven.celsius} Celsius is equivalent to  {oven.farenheit} Farenheit')


100.0 Farenheit is equivalent to  37.78 Celsius
37.78 Celsius is equivalent to  100.0 Farenheit


In [37]:
#Example of Property
class Temperature( object ):
    def fget( self ):
        return round(self.celsius * 9 / 5 + 32,2)
    def fset( self, value ):
        self.celsius= round((float(value)-32) * 5 / 9,2)

    farenheit= property( fget, fset )
    
    def cset( self, value ):
        self.cTemp= float(value)
    def cget( self ):
        return self.cTemp
    
    celsius= property( cget, cset, doc="Celsius temperature")

oven= Temperature()
oven.farenheit= 100
print(f'{oven.farenheit} Farenheit is equivalent to  {oven.celsius} Celsius')
celsius= oven.celsius
oven.celsius = celsius
print(f'{oven.celsius} Celsius is equivalent to  {oven.farenheit} Farenheit')


100.0 Farenheit is equivalent to  37.78 Celsius
37.78 Celsius is equivalent to  100.0 Farenheit


In [36]:
#Example of Property using @property Decorator
class Alphabets:  
    def __init__(self, value):  
        self._value = value  
            
    # getting the values      
    @property
    def value(self):  
        print('Getting value')  
        return self._value  
            
    # setting the values      
    @value.setter  
    def value(self, value):  
        print('Setting value to ' + value)  
        self._value = value  
            
    # deleting the values  
    @value.deleter  
    def value(self):  
        print('Deleting value')  
        del self._value  
    
    
# passing the value  
x = Alphabets('Hello')  
print(x.value)  
print()    
x.value = 'World'
print(x.value)
print()
del x.value 
print(x.value) 

Getting value
Hello

Setting value to World
Getting value
World

Deleting value
Getting value


AttributeError: 'Alphabets' object has no attribute '_value'

#### Q3. What are the key differences in functionality between __getattr__ and __getattribute__, as well as properties and descriptors?
**Ans:** The Key Differences between `__getattr__`, ` __getattribute__`, Properties and Descriptors are:

**`__getattr__`**: Python will call this method whenever we request an attribute that hasn't already been defined.

**`__getattribute__`** : This method will invoked before looking at the actual attributes on the object. Means, if we have `__getattribute__` method in our class, python invokes this method for every attribute regardless whether it exists or not. 

**Properties:** With Properties we can bind getter, setter and delete functions together with an attribute name, using the built-in property function or @property decorator. 

**Descriptor:** With Descriptor we can bind getter, setter and delete functions into a seperate class. we then assign an object of this class to the attribute name in our main class.